# MNIST Handwritten digit classifier in browser

This notebook creates, train and export a model trained in MNIST handwritten digit classifying task, to be imported by tfjs in browser.


install `tensorflowjs`, which is only required in this notebook for saving the model compatible with `tensorflowjs`

In [4]:
!pip install tensorflowjs

    100% |████████████████████████████████| 86.5MB 448kB/s 
    100% |████████████████████████████████| 13.9MB 3.1MB/s 
    100% |████████████████████████████████| 81kB 21.2MB/s 
    100% |████████████████████████████████| 358kB 20.4MB/s 
    100% |████████████████████████████████| 3.0MB 8.3MB/s 
    100% |████████████████████████████████| 61kB 24.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/89/67/41/63cbf0f6ac0a6156588b9587be4db5565f8c6d8ccef98202fc
Successfully built wrapt
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: wrapt 1.10.11
    Uninstalling wrapt-1.10.11:
      Successfully uninstalled wrapt-1.10.11
  Found existing installation: numpy 1.16.2
    Uninstalling numpy-1.16

Packages required for the notebook

In [2]:
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow import keras

print('tensorflow', tf.__version__)
print('tensorflowjs', tfjs.__version__)
print('keras', keras.__version__)

tensorflow 2.0.0-dev20190412
tensorflowjs 1.0.1
keras 2.2.4-tf


### Constants

In [0]:
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1

NUM_EPOCHS = 10

## The model

Let's create the model

In [0]:
def getModel():
    model = keras.Sequential();
    
    # In the first layer of out convolutional neural network we have
    # to specify the input shape. Then we specify some paramaters for
    # the convolution operation that takes place in this layer.
    model.add(keras.layers.Conv2D(
        input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
        kernel_size=5,
        filters=8,
        strides=1,
        activation='relu',
        kernel_initializer='VarianceScaling'
    ))
    
    # The MaxPooling layer acts as a sort of downsampling using max values
    # in a region instead of averaging.
    model.add(keras.layers.MaxPool2D(
        pool_size=(2,2), 
        strides=(2,2)
    ))
    
    # Repeat another Conv2D + MazPooling stack.
    # Note that we have more filters in the convolution.
    model.add(keras.layers.Conv2D(
        kernel_size=5,
        filters=16,
        strides=1,
        activation='relu',
        kernel_initializer='VarianceScaling'
    ))
    
    model.add(keras.layers.MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))
    
    
    # Now we flatten the output from the 2D filters into a 1D vector to prepare
    # it for input into our last layer. This is common practice when feeding
    # higher dimensional data to a final classification output layer.
    model.add(keras.layers.Flatten())
    
    
    # Our last layer is a dense layer which has 10 output units, one for each
    # output class (i.e. 0, 1, 2, 3, 4, 5, 6, 7, 8, 9).
    NUM_OUTPUT_CLASSES = 10;
    model.add(keras.layers.Dense(
        units=NUM_OUTPUT_CLASSES,
        kernel_initializer='VarianceScaling',
        activation='softmax'
    ))
    
    
    # Choose an optimizer, loss function and accuracy matric,
    # then compile and return the model
    optimizer = keras.optimizers.Adam();
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model
  
model = getModel()


## Train the model

In [43]:
from keras.datasets import mnist
from keras.utils import to_categorical

def train(model, data, epochs):
  BATCH_SIZE = 512
  NUM_TRAIN_SAMPLES = 60000
  NUM_TEST_SAMPLES = 10000

  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  
  x_train = x_train.reshape((NUM_TRAIN_SAMPLES, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))
  x_test = x_test.reshape((NUM_TEST_SAMPLES, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))
  
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  
  model.fit(
      x_train,
      y_train,
      batch_size=BATCH_SIZE,
      epochs=NUM_EPOCHS,
      validation_data=(x_test, y_test),
      shuffle=True,
      callbacks=None
  )
  

train(model, mnist, NUM_EPOCHS)


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 22s 363us/sample - loss: 11.4702 - accuracy: 0.5888 - val_loss: 1.7214 - val_accuracy: 0.8089
Epoch 2/10
60000/60000 [==============================] - 22s 363us/sample - loss: 1.2803 - accuracy: 0.8426 - val_loss: 0.8812 - val_accuracy: 0.8784
Epoch 3/10
60000/60000 [==============================] - 22s 363us/sample - loss: 0.7405 - accuracy: 0.8901 - val_loss: 0.6032 - val_accuracy: 0.9054
Epoch 4/10
60000/60000 [==============================] - 22s 364us/sample - loss: 0.5202 - accuracy: 0.9094 - val_loss: 0.4665 - val_accuracy: 0.9193
Epoch 5/10
60000/60000 [==============================] - 22s 362us/sample - loss: 0.3927 - accuracy: 0.9243 - val_loss: 0.3720 - val_accuracy: 0.9283
Epoch 6/10
60000/60000 [==============================] - 22s 362us/sample - loss: 0.3157 - accuracy: 0.9339 - val_loss: 0.3056 - val_accuracy: 0.9352
Epoch 7/10
60000/60000 [===================

### Run the following code to authenticate/authorize your Google Drive for the purpose of mounting in this virtual machine
### we will use this to save the trained model to your GDrive

In [44]:
# Run this to authenticate/authorize your google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Save the model

In [0]:
MODEL_SAVE_PATH_GDRIVE = '/content/gdrive/My Drive/mnist_cnn_tfjs'
tfjs.converters.save_keras_model(model, MODEL_SAVE_PATH_GDRIVE)

### The trained model is now available in your Google Drive in the location `My Drive/mnist_cnn_tfjs`